## Sandbox for Implementing RetinaNet

- start wih pretrined resnet50, maybe from the original paper, or mybe from fastai, doesn't really matter
- then add feature pyramid network and finetune for coco

## General Approach

1. Download COCO
- Build and train U-Net
- Build Feature Pyramid Network (FPN) with ResNet backbone
- Build and train RetinaNet with FPN and object detection subnets

In [39]:
%reload_ext autoreload
%autoreload 2

In [40]:
from fastai.vision import *
from fastai.datasets import *
from fastai.callbacks import *

## COCO

In [41]:
path = untar_data(URLs.COCO_SAMPLE)
path.ls()

[PosixPath('/home/sudosharma/projects/data/coco_sample/annotations'),
 PosixPath('/home/sudosharma/projects/data/coco_sample/train_sample')]

In [43]:
tfms = get_transforms()
data = ImageDataBunch.from_folder(path, train='train_sample', ds_tfms=tfms, valid_pct=0.2)

You can deactivate this warning by passing `no_check=True`.


/home/sudosharma/miniconda3/envs/wilbur/lib/python3.7/site-packages/fastai/basic_data.py:257: UserWarning: It's not possible to collate samples of your dataset together in a batch.
Shapes of the inputs/targets:
[[torch.Size([3, 640, 480]), torch.Size([3, 441, 640]), torch.Size([3, 427, 640]), torch.Size([3, 640, 480]), torch.Size([3, 480, 640]), torch.Size([3, 375, 500]), torch.Size([3, 480, 640]), torch.Size([3, 640, 361]), torch.Size([3, 338, 640]), torch.Size([3, 427, 640]), torch.Size([3, 427, 640]), torch.Size([3, 478, 640]), torch.Size([3, 612, 612]), torch.Size([3, 500, 332]), torch.Size([3, 427, 640]), torch.Size([3, 480, 640]), torch.Size([3, 421, 640]), torch.Size([3, 640, 456]), torch.Size([3, 375, 500]), torch.Size([3, 480, 640]), torch.Size([3, 425, 640]), torch.Size([3, 428, 640]), torch.Size([3, 427, 640]), torch.Size([3, 640, 480]), torch.Size([3, 640, 427]), torch.Size([3, 640, 427]), torch.Size([3, 426, 640]), torch.Size([3, 640, 427]), torch.Size([3, 563, 640]), torc

## U-Net

In [11]:
def conv(ic, oc, k=3, s=1, actn=True):
    layers = [nn.Conv2d(ic, oc, k, stride=s, padding=0)]
    if actn: layers.append(nn.ReLU(True))
    return nn.Sequential(*layers)

def conv_block(ic, oc): return nn.Sequential(conv(ic, oc), conv(oc, oc), nn.MaxPool2d(2, stride=2))
def contraction(cs): return nn.Sequential(*[conv_block(cs[i], cs[i+1]) for i in range(len(cs)-1)])

class SaveMap():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
        
    def hook_fn(self, module, input, output): self.map.append(out)
    def close(self): self.hook.remove()

In [35]:
cs = [3, 64, 128, 256, 512, 1024]
model = contraction(cs)
model[0][1]

Sequential(
  (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU(inplace)
)